In [3]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install -q openai-whisper ffmpeg

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 1.6 MB/s eta 0:00:00
     ━━━━━━

In [34]:
import whisper
model = whisper.load_model("medium")

100%|██████████████████████████████████████| 1.42G/1.42G [00:14<00:00, 107MiB/s]


In [35]:
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode
from io import BytesIO
#!pip install pydub
from pydub import AudioSegment

JAVASCRIPT_RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""


def record(sec=3):
  print("Start recording...")
  display(Javascript(JAVASCRIPT_RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  audio = AudioSegment.from_file(BytesIO(b))
  return audio
def speech_to_text(sec=3):
  print("Start recording...")
  display(Javascript(JAVASCRIPT_RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  with open('/content/audio.wav',"wb") as f:
    f.write(b)
  print("Finish recording, and transfer to text")
  result = model.transcribe("/content/audio.wav",language ="vietnamese")
  return result


In [28]:
record()

Start recording...


<IPython.core.display.Javascript object>

In [36]:
speech_to_text()

Start recording...


<IPython.core.display.Javascript object>

Finish recording, and transfer to text


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


{'text': ' Tôi thích một người con gái tuyệt vời!',
 'segments': [{'id': 0,
   'seek': 0,
   'start': 0.0,
   'end': 3.0,
   'text': ' Tôi thích một người con gái tuyệt vời!',
   'tokens': [50364,
    43345,
    258,
    30839,
    15486,
    15898,
    416,
    290,
    28249,
    2604,
    88,
    21931,
    371,
    21024,
    0,
    50514],
   'temperature': 0.0,
   'avg_logprob': -0.35000181198120117,
   'compression_ratio': 0.8928571428571429,
   'no_speech_prob': 0.4138481914997101}],
 'language': 'vietnamese'}

In [38]:
!pip install -q transformers pydub

In [ ]:
from transformers import pipeline
transcriber = pipeline("automatic-speech-recognition", model="vinai/PhoWhisper-large",device = "cuda")

In [2]:
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode
from io import BytesIO
from pydub import AudioSegment

JAVASCRIPT_RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""


def speech_to_text_pho(sec=5):
  print("Start recording...")
  display(Javascript(JAVASCRIPT_RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  with open('/content/audio.wav',"wb") as f:
    f.write(b)
  print("Finish recording. Send to model")
  result = transcriber("/content/audio.wav")
  return result["text"]


In [4]:
speech_to_text_pho()

Start recording...


<IPython.core.display.Javascript object>

Finish recording. Send to model


'tuy nhiên không phải mỗi người đều có thể đạt được thành công.'